## *Fixed - gaze task*
# N2 and P2 peak - Visualizations

In [1]:
%matplotlib ipympl
import os.path as op
import mne
from mayavi import mlab
import matplotlib.pyplot as plt

In [2]:
# Set up dirs needed for all visualizations
experiment_dir = '../fixed_gaze/'
conditions = ['phase_rand', 'armenian', 'normal']

## Sensor space

In [4]:
# Read average evoked files from all three conditions 
average_evoked = []
for it, condition in enumerate(conditions):
    evoked_dir = op.join(experiment_dir, condition, 'evoked')
    evoked_file =  op.join(evoked_dir, 'average_' + condition + '-ave.fif')
    average_evoked.append(mne.read_evokeds(evoked_file, condition = 0, baseline=None, proj=False,verbose=False).crop(-0.050,None))

In [5]:
# Plot evoked potentials of all three conditions, from stimulus onset   
fig0, axs = plt.subplots(3,1, sharex=True, sharey = False, figsize = (12.0,5.8))
for it, condition in enumerate(conditions):
    average_evoked[it].plot(spatial_colors=True, axes = axs[it])
    axs[it].set_title(condition.capitalize())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Plot only N2 and P2 time window
fig1, axs = plt.subplots(1,3, sharex=False, sharey = True, figsize = (12.0,5.8), squeeze = True)
for it, condition in enumerate(conditions):
    average_evoked[it].crop(0.210,0.340) # change time-window here
    average_evoked[it].plot(spatial_colors=False, axes = axs[it])
    axs[it].set_title(condition.capitalize())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Source space

### Time-line for each condition

In [ ]:
noise_cov = 'with_non_reg_noise_cov'
method = 'MNE'
views = ['caudal', 'lateral']
hemis = ['both', 'lh'] # looped together with views, as a tuple

# Select time points to show
time_points = [0.215, 0.235, 0.245, 0.255, 0.265, 0.280, 0.290, 0.300]
time_sections = [0,2,4,6] # index of first subplot in figure, 2 images are shown per figure

mlab.options.offscreen = True
for condition in conditions:
    average_stc_file = op.join(experiment_dir, condition, noise_cov, method, condition + '_average')
    average_stc = mne.read_source_estimate(average_stc_file, 'fsaverage')
    for t in time_sections:
        for view, hemi in zip(views, hemis):
            fig1, axs1 = plt.subplots(1,2, sharex=True, sharey = True)
            fig1.suptitle(condition.capitalize() + ' condition')
            surfer_kwargs = dict(hemi=hemi, clim=dict(kind='value', lims=[6.5e-12, 7.0e-12, 1.6e-11 ]),
                                 views=view, time_unit='s', smoothing_steps=5, colorbar = False, size = 500, time_label = None)
            for col,time_point in enumerate(time_points[t:(t+2)]):
                brain = average_stc.plot(initial_time = time_point, **surfer_kwargs)
                img = mlab.screenshot()
                mlab.close()
                axs1[col].imshow(img)
                axs1[col].set_axis_off()
                axs1[col].set_title(str(int(time_point*1000))+ ' ms')


### Compare images between conditions at each time point

In [ ]:
noise_cov = 'with_non_reg_noise_cov'
method = 'MNE'
views = ['caudal', 'lateral']
hemis = ['both', 'lh'] # looped together with views, as a tuple

time_points = [0.220, 0.230, 0.240, 0.250, 0.260] # Time points of interest

mlab.options.offscreen = True
for time_point in time_points:
    for view, hemi in zip(views, hemis):
        surfer_kwargs = dict(hemi=hemi, clim=dict(kind='value', lims=[6.5e-12, 7.0e-12, 1.6e-11 ]),
                             views=view, time_unit='s', smoothing_steps=5, colorbar = False, size = 700, time_label = None)
        fig, axs = plt.subplots(1,3, sharex=True, sharey = True)
        fig.suptitle('Time point: ' + str(int(time_point*1000)) + ' ms')
        for col, condition in enumerate(conditions):
            average_stc_file = op.join(experiment_dir, condition, noise_cov, method, condition + '_average')
            average_stc = mne.read_source_estimate(average_stc_file, 'fsaverage')
            brain = average_stc.plot(initial_time = time_point, **surfer_kwargs)
            img = mlab.screenshot(antialiased = True)
            mlab.close()
            axs[col].imshow(img)
            axs[col].set_axis_off()
            axs[col].set_title(condition.capitalize())


### Interactive figure with time viewer

In [ ]:
cond = 'normal'
method = 'MNE'
lims = [6.5e-12, 7.0e-12, 1.6e-11 ]
noise_cov = 'with_non_reg_noise_cov'
views = ['lat','caudal','ventral']
mlab.options.offscreen = False

stc_file = op.join(experiment_dir, cond, noise_cov, method, cond + '_average')
stc = mne.read_source_estimate(stc_file, 'fsaverage')
tview_kwargs = dict(hemi='split', clim=dict(kind='value', lims=lims),
                     views=views, time_unit='s', smoothing_steps=5, colorbar = True, size = 1400, 
                     initial_time = 0.260, time_viewer = True)
brain = stc.plot(**tview_kwargs)

### Create and save video

In [ ]:
cond = 'armenian'
tmin = 0.210
tmax = 0.300
method = 'dSPM'
lims = [6.5, 7, 11 ]
noise_cov = 'with_non_reg_noise_cov'
views = ['lat','caudal','ventral']
video_file = 'N2-P2_' + cond + '_'+ method +'.mov'

mlab.options.offscreen = False
stc_file = op.join(experiment_dir, cond, noise_cov, method, cond + '_average')
stc = mne.read_source_estimate(stc_file, 'fsaverage')
video_kwargs = dict(hemi='split', clim=dict(kind='value', lims=lims),
                     views=views, time_unit='s', smoothing_steps=5, colorbar = True, size = 1600)
brain = stc.plot(**video_kwargs)
brain.save_movie(video_file, tmin=tmin, tmax= tmax, time_dilation = 200, framerate = 24)
mlab.close()

## Condition contrasts in source space

In [ ]:
# Contrast function
def create_contrast_stc(stc1, stc2, tmin = None, tmax = None):
    
    # Read files if input is string
    if isinstance(stc1, str):
        stc1 = mne.read_source_estimate(stc1, 'fsaverage')
    if isinstance(stc2, str):
        stc2 = mne.read_source_estimate(stc2, 'fsaverage')
    # Crop
    stc1.crop(tmin, tmax)
    stc2.crop(tmin, tmax)
    # Create contrast
    contrast = stc1.data - stc2.data
    stc1.data = contrast
    # Return stc with contrast data
    return stc1

### Interactive figure with time-viewer

In [ ]:
tmin = 0.0
tmax = 0.300
method = 'MNE'
noise_cov = 'with_non_reg_noise_cov'
# STC files
phase_rand_file = op.join(experiment_dir, conditions[0], noise_cov, method, conditions[0] + '_average')
armenian_file = op.join(experiment_dir, conditions[1], noise_cov, method, conditions[1] + '_average')
normal_file = op.join(experiment_dir, conditions[2], noise_cov, method, conditions[2] + '_average')
contrast = create_contrast_stc(normal_file, armenian_file, tmin = tmin, tmax = tmax) # add conditions to compare here

lims = [1.5e-12, 1.5e-12, 6.5e-12]

mlab.options.offscreen = False
contrast_kwargs = dict(hemi='split', views=['lateral', 'caudal', 'ventral'], clim=dict(kind='value', pos_lims=lims),
                       initial_time= tmin, time_unit='s', size=(1200, 1200), smoothing_steps=5, time_viewer = True)

brain = contrast.plot(**contrast_kwargs)